In [3]:
#Importing Necessary Liberaries

import pandas as pd
import numpy as np
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
# print(openai.api_key) 

#Reading  defects csv file
jira_tms_data = pd.read_csv("NetCracker TMS 2023-07-22T08_01_52+0100.csv")

C:\Users\mach1220\AppData\Local\Temp\1\ipykernel_23664\1237754485.py:14: DtypeWarning: Columns (25,26,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  jira_tms_data = pd.read_csv("NetCracker TMS 2023-07-22T08_01_52+0100.csv")


# Document Loading

In [27]:
#Loading one or more documents
from langchain.document_loaders import PyPDFLoader
# loader = PyPDFLoader("MachineLearning-Lecture01.pdf")
loader = PyPDFLoader("MG.Replicated_Database_10.0.6.0.pdf")
pages = loader.load()

In [28]:
type(pages)

list

In [19]:
pages[4]

Document(page_content=' \nReplicated Database v.10.0.6.0.01 5 \nMaintenance Guide  \nReplicated Database \nIntroduction to Replicated Database Maintenance \nThis document is intended for system administrators , backup operators, and IT staff \nwho maintain the NetCracker system. \nThis document describes the following maintenance p rocedures for the NetCracker \nReplicated Database (RDB) component: \n• Pre-installation steps \nThese steps are recommended before any new installa tion of RDB. For details, \nsee "Pre-Installation Steps." \n• RDB component installation/upgrade \nThis section provides high level instructions for t he following cases:  \n• You need to install the RDB component on a new Net Cracker server \n• You have an old version of the RDB component on a NetCracker server \nand you need to upgrade this version to a new one. \n• You need to extend existing RDB configuration by i nstalling new RDB \ninstance. \n• You need to uninstall RDB instance from existing R DB configu

## Document splitting details

`RecursiveCharacterTextSplitter` is recommended for generic text. 

In [21]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    separators=["\n\n", "\n", "(?<=\.)", " ", ""]
)

In [22]:
splits = text_splitter.split_documents(pages)

In [23]:
len(splits)

116

In [26]:
splits[10:20]

[Document(page_content='Replicated Database v.10.0.6.0.01 5 \nMaintenance Guide  \nReplicated Database \nIntroduction to Replicated Database Maintenance \nThis document is intended for system administrators , backup operators, and IT staff \nwho maintain the NetCracker system. \nThis document describes the following maintenance p rocedures for the NetCracker \nReplicated Database (RDB) component: \n• Pre-installation steps \nThese steps are recommended before any new installa tion of RDB. For details,', metadata={'source': 'MG.Replicated_Database_10.0.6.0.pdf', 'page': 4}),
 Document(page_content='see "Pre-Installation Steps." \n• RDB component installation/upgrade \nThis section provides high level instructions for t he following cases:  \n• You need to install the RDB component on a new Net Cracker server \n• You have an old version of the RDB component on a NetCracker server \nand you need to upgrade this version to a new one. \n• You need to extend existing RDB configuration by i n

## Embeddings

Let's take our splits and embed them.

In [29]:
# Function to query the ChatGPT model with JIRA data
def query_chatbot(query, jira_data):
    input_text = f"User Query: {query}\nJIRA Data:\n{str(jira_data)}\nChatGPT:"

    # Call the OpenAI API to get the response
    response = openai.Completion.create(
        engine="text-davinci-003",  # Choose the appropriate GPT model -text-davinci-003
        prompt=input_text,
        max_tokens=500  # Adjust as needed based on the response length you want
    )

    return response['choices'][0]['text']

In [34]:
def main():
    # Read JIRA data from Excel file
    jira_data_df = jira_tms_data.head(5)

    # Convert JIRA data DataFrame to a dictionary
    jira_data = jira_data_df.to_dict(orient='records')

    while True:
        user_query = input("You: ").strip().lower()

        if user_query == "exit":
            print("Chatbot: Goodbye!")
            break

#         response = query_chatbot(user_query, jira_data)
        response = query_chatbot(user_query, pages[4:9])
        print(f"Chatbot: {response}")

In [35]:
if __name__ == '__main__':
    main()

You: what is pre-installation step of RDB
Chatbot: 
The pre-installation steps for RDB include calculating the free space required for installation, using installation parameters, and adding snapshots.
You: which script to run to check the free space
Chatbot:  Run the RDB_recommended_tablespa сes_size.sql script to check the free space.
You: exit
Chatbot: Goodbye!


In [1]:
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers

def count_tokens(prompt, tokenizer):
    tokens = tokenizer.encode(prompt).tokens
    return len(tokens)

# Your prompt as a string
prompt = "what is the total count of TMS which are in open status"

# Create and configure the tokenizer (you can choose a model according to your needs)
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
tokenizer.decoder = decoders.ByteLevel()
# tokenizer.post_processor = processors.ByteLevel(trim_offsets=True)
tokenizer.enable_truncation(max_length=4096)

# Measure the context length
context_length = count_tokens(prompt, tokenizer)
print("Context length:", context_length)



Context length: 0
